In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

from textblob import TextBlob, Word

import re

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

Edited version of AOC to make it more organized

In [2]:
pd.set_option('display.max_colwidth', 400)  # or 199

Now, preprocessing (using same code as for kickstarter as baseline, come back here to tweak later)

Look into https://pypi.org/project/tweet-preprocessor/ for tweet processing later!

In [3]:
nlp = spacy.load('en', disable=['parser', 'ner'])
# nlp = spacy.load('en')

In [4]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #lemmatize with Spacy
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    
    return text

In [5]:
tweet_stopwords = stopwords.words('english') + \
    ['rt', 'https', 'http', 'amp', 'via', 'one', 'around', 'would', 'let', 'could', 'going', 'like', 
     'get', 'may', 'says', 'say', 'make', 'based', 'even', 'another', 'completely', 'thanks', 'way', 
     'find', 'used', 'thing', '2019', 'see', 'need', 'know', 'knows', 'think', 'thinks', 'take', 'new', 
     'day', 'days', 'john', 'mccain', 'johnmccain']

In [6]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [7]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_0.pickle', 'rb') as f:
    df_0 = pickle.load(f)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_1.pickle', 'rb') as f:
    df_1 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_3.pickle', 'rb') as f:
    df_3 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_4.pickle', 'rb') as f:
    df_4 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_5.pickle', 'rb') as f:
    df_5 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_6.pickle', 'rb') as f:
    df_6 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_7.pickle', 'rb') as f:
    df_7 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_8.pickle', 'rb') as f:
    df_8 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_9.pickle', 'rb') as f:
    df_9 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_10.pickle', 'rb') as f:
    df_10 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_11.pickle', 'rb') as f:
    df_11 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_12.pickle', 'rb') as f:
    df_12 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_13.pickle', 'rb') as f:
    df_13 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_14.pickle', 'rb') as f:
    df_14 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_15.pickle', 'rb') as f:
    df_15 = pickle.load(f)


In [8]:
df_0['final_text'] = np.where((df_0['retweet_text'].isnull() == False), df_0['retweet_text'], df_0['main_text'])
df_1['final_text'] = np.where((df_1['retweet_text'].isnull() == False), df_1['retweet_text'], df_1['main_text'])
df_3['final_text'] = np.where((df_3['retweet_text'].isnull() == False), df_3['retweet_text'], df_3['main_text'])
df_4['final_text'] = np.where((df_4['retweet_text'].isnull() == False), df_4['retweet_text'], df_4['main_text'])
df_5['final_text'] = np.where((df_5['retweet_text'].isnull() == False), df_5['retweet_text'], df_5['main_text'])
df_6['final_text'] = np.where((df_6['retweet_text'].isnull() == False), df_6['retweet_text'], df_6['main_text'])
df_7['final_text'] = np.where((df_7['retweet_text'].isnull() == False), df_7['retweet_text'], df_7['main_text'])
df_8['final_text'] = np.where((df_8['retweet_text'].isnull() == False), df_8['retweet_text'], df_8['main_text'])
df_9['final_text'] = np.where((df_9['retweet_text'].isnull() == False), df_9['retweet_text'], df_9['main_text'])
df_10['final_text'] = np.where((df_10['retweet_text'].isnull() == False), df_10['retweet_text'], df_10['main_text'])
df_11['final_text'] = np.where((df_11['retweet_text'].isnull() == False), df_11['retweet_text'], df_11['main_text'])
df_12['final_text'] = np.where((df_12['retweet_text'].isnull() == False), df_12['retweet_text'], df_12['main_text'])
df_13['final_text'] = np.where((df_13['retweet_text'].isnull() == False), df_13['retweet_text'], df_13['main_text'])
df_14['final_text'] = np.where((df_14['retweet_text'].isnull() == False), df_14['retweet_text'], df_14['main_text'])
df_15['final_text'] = np.where((df_15['retweet_text'].isnull() == False), df_15['retweet_text'], df_15['main_text'])

RUN ALL BELOW ONCE MARVEL DONE

In [9]:
df_0['tweet_processed'] = df_0['final_text'].apply(lambda x:pre_process(x))
df_1['tweet_processed'] = df_1['final_text'].apply(lambda x:pre_process(x))
df_3['tweet_processed'] = df_3['final_text'].apply(lambda x:pre_process(x))
df_4['tweet_processed'] = df_4['final_text'].apply(lambda x:pre_process(x))
df_5['tweet_processed'] = df_5['final_text'].apply(lambda x:pre_process(x))
df_6['tweet_processed'] = df_6['final_text'].apply(lambda x:pre_process(x))
df_7['tweet_processed'] = df_7['final_text'].apply(lambda x:pre_process(x))
df_8['tweet_processed'] = df_8['final_text'].apply(lambda x:pre_process(x))
df_9['tweet_processed'] = df_9['final_text'].apply(lambda x:pre_process(x))
df_10['tweet_processed'] = df_10['final_text'].apply(lambda x:pre_process(x))
df_11['tweet_processed'] = df_11['final_text'].apply(lambda x:pre_process(x))
df_12['tweet_processed'] = df_12['final_text'].apply(lambda x:pre_process(x))
df_13['tweet_processed'] = df_13['final_text'].apply(lambda x:pre_process(x))
df_14['tweet_processed'] = df_14['final_text'].apply(lambda x:pre_process(x))
df_15['tweet_processed'] = df_15['final_text'].apply(lambda x:pre_process(x))

In [10]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_0_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_0, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_1_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_1, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_3_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_3, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_4_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_4, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_5_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_5, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_6_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_6, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_7_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_7, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_8_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_8, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_9_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_9, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_10_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_10, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_11_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_11, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_12_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_12, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_13_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_13, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_14_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_14, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/preproccessed_text/df_15_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_15, to_write)



In [11]:
X_0 = df_1['tweet_processed']
X_1 = df_1['tweet_processed']
X_3 = df_3['tweet_processed']
X_4 = df_4['tweet_processed']
X_5 = df_5['tweet_processed']
X_6 = df_6['tweet_processed']
X_7 = df_7['tweet_processed']
X_8 = df_8['tweet_processed']
X_9 = df_9['tweet_processed']
X_10 = df_10['tweet_processed']
X_11 = df_11['tweet_processed']
X_12 = df_12['tweet_processed']
X_13 = df_13['tweet_processed']
X_14 = df_14['tweet_processed']
X_15 = df_15['tweet_processed']

In [12]:
tfidf_0 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_1 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_3 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_4 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_5 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_6 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_7 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_8 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_9 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_10 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_11 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_12 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_13 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_14 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_15 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))


Community_0

In [13]:
bag_of_words_0 = tfidf_0.fit_transform(X_0)
lsa_0 = TruncatedSVD(20)
doc_topic_0 = lsa_0.fit_transform(bag_of_words_0)
lsa_0.explained_variance_ratio_

array([0.02001522, 0.02271835, 0.0225802 , 0.01939186, 0.01878646,
       0.01759758, 0.01712557, 0.01368664, 0.01378972, 0.0133207 ,
       0.01290244, 0.01246944, 0.01199284, 0.01176239, 0.01138351,
       0.01022373, 0.00905451, 0.00846325, 0.00822894, 0.0082056 ])

In [14]:
display_topics(lsa_0, tfidf_0.get_feature_names(), 10)


Topic  0
pron, aide, former aide, trump lie, lie, former, trump, senator, president, today

Topic  1
aide, former aide, trump lie, former, lie, replace effort, go support, mind nope, effort change, mislead wh

Topic  2
abandon pron, pron lose, abandon, lose pron, lose, pron friend, friend, life lindseygrahamsc, pow offer, apart abandon

Topic  3
cathedral, funeral, national cathedral, national, funeral pron, kind funeral, president approve, pron want, want president, approve

Topic  4
billion, cut trillion, trillion medicaid, medicare billion, billion medicare, billion social, medicaid billion, budget cut, medicaid, divert attention

Topic  5
american, smear, american hero, hero, soon introduce, legislation name, forward soon, pron legislation, building american, introduce pron

Topic  6
forward soon, soon introduce, legislation name, pron legislation, building american, introduce pron, senate russell, look forward, russell building, name senate

Topic  7
pron, co, well, week, country

Community_1

In [15]:
bag_of_words_1 = tfidf_1.fit_transform(X_1)
lsa_1 = TruncatedSVD(20)
doc_topic_1 = lsa_1.fit_transform(bag_of_words_1)
lsa_1.explained_variance_ratio_

array([0.02001541, 0.02271841, 0.02258013, 0.01939218, 0.01878621,
       0.01759694, 0.01712559, 0.01368559, 0.01378718, 0.01331913,
       0.01290128, 0.01246776, 0.01200079, 0.01175976, 0.01137744,
       0.01023973, 0.00904567, 0.00844509, 0.0083269 , 0.00822608])

In [16]:
display_topics(lsa_1, tfidf_1.get_feature_names(), 10)


Topic  0
pron, aide, former aide, trump lie, lie, former, trump, senator, president, today

Topic  1
aide, former aide, trump lie, former, lie, mind nope, go support, effort change, replace effort, mislead wh

Topic  2
abandon pron, pron lose, abandon, lose pron, lose, pron friend, friend, pow offer, life lindseygrahamsc, silence bully

Topic  3
cathedral, funeral, national cathedral, national, funeral pron, kind funeral, president approve, pron want, want president, approve

Topic  4
billion, trillion medicaid, cut trillion, medicare billion, billion medicare, billion social, medicaid billion, budget cut, medicaid, divert attention

Topic  5
american, smear, american hero, hero, legislation name, forward soon, soon introduce, pron legislation, building american, introduce pron

Topic  6
forward soon, legislation name, soon introduce, pron legislation, building american, introduce pron, senate russell, look forward, russell building, name senate

Topic  7
pron, co, well, week, country

Community_3

In [17]:
bag_of_words_3 = tfidf_3.fit_transform(X_3)
lsa_3 = TruncatedSVD(20)
doc_topic_3 = lsa_3.fit_transform(bag_of_words_3)
lsa_3.explained_variance_ratio_

array([0.14410814, 0.04251032, 0.03159695, 0.02312115, 0.02288691,
       0.01860273, 0.01648849, 0.01666266, 0.01376408, 0.01381197,
       0.0120043 , 0.01056348, 0.01020041, 0.00920348, 0.00905875,
       0.0080828 , 0.00781501, 0.00737169, 0.00720637, 0.00695077])

In [18]:
display_topics(lsa_3, tfidf_3.get_feature_names(), 10)


Topic  0
smear, pron, trump, therefore repugnant, continuously maliciously, answer smear, repugnant donald, smear attack, longer answer, maliciously

Topic  1
pron, whether pron, whether, pron friend, friend, pron pron, president, pron honor, honor, serve

Topic  2
disability order, also follow, behavior president, order avoid, attack depart, focus behavior, serve attack, president fake, example focus, future pron

Topic  3
pron, country, put pron, put, towards family, direct towards, less attack, nearly give, family wrong, pron less

Topic  4
billion, billion social, medicare billion, pron budget, budget cut, budget, medicaid billion, medicaid, attention fact, cut trillion

Topic  5
cathedral, funeral, president, funeral pron, president approve, kind funeral, want president, pron want, approve, national cathedral

Topic  6
aide, tomorrow ever, legacy tarnish, integrity conviction, tarnish realdonaldtrump, today tomorrow, conviction service, realdonaldtrump today, honor integrity, man

Community_4

In [19]:
bag_of_words_4 = tfidf_4.fit_transform(X_4)
lsa_4 = TruncatedSVD(20)
doc_topic_4 = lsa_4.fit_transform(bag_of_words_4)
lsa_4.explained_variance_ratio_

array([0.03759144, 0.02011964, 0.02020406, 0.01739594, 0.01489511,
       0.01370668, 0.01297365, 0.01294088, 0.01261985, 0.01208543,
       0.0114064 , 0.01061825, 0.00989825, 0.00871479, 0.00777381,
       0.00779426, 0.00762062, 0.00744792, 0.00682538, 0.00638487])

In [20]:
display_topics(lsa_4, tfidf_4.get_feature_names(), 10)


Topic  0
pron, obsess, pron pron, co, great man, great, man, father sen, obsess great, pron weekend

Topic  1
pron, president, funeral, funeral pron, hero, pron attention, disability order, serve attack, example focus, president fake

Topic  2
president future, also follow, focus behavior, example focus, depart war, behavior president, serve attack, president fake, attack depart, disability order

Topic  3
pron pron, pron, hurt, criticize, criticize pron, weekend obsess, never great, man co, pron spend, tweet

Topic  4
republican, fire back, back trump, cheapen white, silence complicity, complicity republican, party enable, trump cheapen, house presidency, cheapen

Topic  5
aide, former aide, trump lie, lie, former, surprised trump, go support, replace effort, mislead wh, effort change

Topic  6
self, patriotic honorable, courageous patriotic, efface self, empathetic, sacrifice empathetic, drive duty, empathetic drive, duty family, heroic courageous

Topic  7
dominate news, hilarious 

Community_5

In [21]:
bag_of_words_5 = tfidf_5.fit_transform(X_5)
lsa_5 = TruncatedSVD(20)
doc_topic_5 = lsa_5.fit_transform(bag_of_words_5)
lsa_5.explained_variance_ratio_

array([0.09433127, 0.05377971, 0.03691388, 0.02320218, 0.02593193,
       0.02328587, 0.02320208, 0.01938591, 0.01710514, 0.01361791,
       0.0134561 , 0.01235638, 0.01213643, 0.01169754, 0.01064588,
       0.01025818, 0.00956481, 0.00866903, 0.00850827, 0.0082233 ])

In [22]:
display_topics(lsa_5, tfidf_5.get_feature_names(), 10)


Topic  0
pron, ejtb, axp, complain come, ejtb axp, co ejtb, nice funeral, pron hold, pron nice, hold pron

Topic  1
tempt ghost, tempt, ghost twitter, trump bait, twitter account, bait, account trump, account, twitter, ghost

Topic  2
behind motorcade, exhume corpse, ramp, drag senator, corpse behind, motorcade co, motorcade, attack drag, ramp attack, exhume

Topic  3
pron, decide, court, number, pron pron, president, politic, pron president, hat, unite supreme

Topic  4
fight, gravelgang co, fight die, co vo, american youth, vo, bernie disagree, meet problem, pron strongly, peace gravelgang

Topic  5
ask, hat, pron, pron want, president hat, hat white, hat pron, policy impact, impact people, impact

Topic  6
hat, hat white, president hat, hat pron, white nationalist, nationalist, pron hat, white, president, kill

Topic  7
pron pron, talk, pron, til, pron major, talk pee, die nothing, medium star, kingmaker conservative, til die

Topic  8
pron lifeless, counterproductive, flail, lifel

Community_6

In [23]:
bag_of_words_6 = tfidf_6.fit_transform(X_6)
lsa_6 = TruncatedSVD(20)
doc_topic_6 = lsa_6.fit_transform(bag_of_words_6)
lsa_6.explained_variance_ratio_

array([0.40774028, 0.0236948 , 0.0112205 , 0.00990356, 0.00928404,
       0.00907323, 0.0072282 , 0.00632771, 0.00609492, 0.00583153,
       0.00551634, 0.00545411, 0.00522067, 0.00491896, 0.0046174 ,
       0.00441877, 0.00388423, 0.0038702 , 0.00364844, 0.00362795])

In [24]:
display_topics(lsa_6, tfidf_6.get_feature_names(), 10)


Topic  0
pron, president, president future, attack depart, order avoid, senator also, behavior president, example focus, also follow, pron attention

Topic  1
stuff co, state retweet, wztlcvztnx, standard stuff, attack daughter, co wztlcvztnx, daughter standard, retweet attack, standard, stuff

Topic  2
trump, pron, co, funeral, pron pron, never, give, sen, thank, aide

Topic  3
russell building, russell, building, isakson, effort rename, schumer play, politic effort, play politic, johnny, johnny isakson

Topic  4
aide, former aide, trump lie, lie, former, replace, repeal replace, repeal, go support, replace effort

Topic  5
smear, aide, former aide, trump lie, answer smear, maliciously, apologize hard, must stop, smear attack, therefore

Topic  6
funeral, cathedral, kind funeral, funeral pron, pron want, president approve, want, want president, kind, approve

Topic  7
american hero, american, senate russell, look forward, soon introduce, forward soon, pron legislation, legislation na

Community_7

In [25]:
bag_of_words_7 = tfidf_7.fit_transform(X_7)
lsa_7 = TruncatedSVD(20)
doc_topic_7 = lsa_7.fit_transform(bag_of_words_7)
lsa_7.explained_variance_ratio_

array([0.09679411, 0.06586066, 0.04968053, 0.02144779, 0.01794947,
       0.018559  , 0.01781719, 0.01778743, 0.01716122, 0.01661907,
       0.01629482, 0.01551139, 0.01526644, 0.01205663, 0.01080944,
       0.00947872, 0.00904783, 0.00864707, 0.00816903, 0.00761576])

In [26]:
display_topics(lsa_7, tfidf_7.get_feature_names(), 10)


Topic  0
graham nothing, unseemly trump, today always, denigrate widow, always denigrate, widow lindsay, anything unseemly, trump constantly, constantly attack, denigrate

Topic  1
haunt sadden, wonder implication, petty insecurity, help expose, implication, sadden infuriate, expose pron, implication guilty, poetic, poetic justice

Topic  2
disgusting ananavarro, plain disgusting, hero plain, dead national, indecent immoral, abominable inhumane, sen abominable, everybody condemn, co wr, wr lcgmm

Topic  3
aide, pron, former aide, trump lie, lie, former, trump, happen, senator, support

Topic  4
pron, president, funeral, country, attention, focus, man, late senator, billion, trump

Topic  5
want, happen scrooge, resign turn, scrooge, holiday turkey, holiday, turkey, turkey pron, generous good, scrooge instead

Topic  6
self, sacrifice, man, honorable self, efface, empathetic, drive duty, patriotic honorable, president disparage, understand president

Topic  7
self, empathetic, patrioti

Community_8

In [27]:
bag_of_words_8 = tfidf_8.fit_transform(X_8)
lsa_8 = TruncatedSVD(20)
doc_topic_8 = lsa_8.fit_transform(bag_of_words_8)
lsa_8.explained_variance_ratio_

array([0.15442106, 0.1386269 , 0.04714734, 0.04132887, 0.03427103,
       0.03266857, 0.02159418, 0.02064273, 0.01373231, 0.01279388,
       0.01249282, 0.00771677, 0.0079162 , 0.00737697, 0.00723962,
       0.0060897 , 0.00591213, 0.00545161, 0.00499105, 0.00471164])

In [28]:
display_topics(lsa_8, tfidf_8.get_feature_names(), 10)


Topic  0
president seriously, seriously stop, stop talk, seriously, stop, talk senator, mr president, mr, talk, senator

Topic  1
self, empathetic drive, heroic courageous, courageous patriotic, duty family, drive duty, efface self, friend heroic, sacrifice empathetic, efface

Topic  2
pron, reason talk, pron poltical, enemy mr, poltical enemy, poltical, pass pron, pron pass, pron pron, enemy

Topic  3
hero, pron, pron life, pron country, life, patriot, american hero, country, american, lot never

Topic  4
disagree lot, reputation war, lot never, anything patriot, feel tear, willing lay, hero particularly, idea president, country idea, patriot willing

Topic  5
every, patriot genuine, rare patriot, memory continue, every miss, blessing serve, alongside, continue remind, nation sustain, serve alongside

Topic  6
hit, late, attack, pron, president, man, pron hit, hit back, mark first, repellent

Topic  7
depart war, focus behavior, attack depart, order avoid, also follow, pron attention

Community_9

In [29]:
bag_of_words_9 = tfidf_9.fit_transform(X_9)
lsa_9 = TruncatedSVD(20)
doc_topic_9 = lsa_9.fit_transform(bag_of_words_9)
lsa_9.explained_variance_ratio_

array([0.03039051, 0.04054664, 0.03519066, 0.03459796, 0.02988777,
       0.02932804, 0.0174524 , 0.01723368, 0.0163917 , 0.01495337,
       0.01307522, 0.01302869, 0.01211169, 0.01195712, 0.01076811,
       0.01068463, 0.01024478, 0.00976488, 0.00946673, 0.00926948])

In [30]:
display_topics(lsa_9, tfidf_9.get_feature_names(), 10)


Topic  0
pron, pron pron, trump, rino, trump pron, medium, bad, president, donald trump, donald

Topic  1
fbi, december senator, agent bill, confirm fbi, copy first, priestap confirm, fbi receive, receive copy, counterintelligence fbi, senior counterintelligence

Topic  2
fake, medium, indeed prove, paper last, annapolis send, court paper, prove court, print election, medium hop, dem work

Topic  3
donald trump, donald, medium, hate donald, use advance, racist divisive, divisive voice, advance narrative, voice medium, narrative trump

Topic  4
young, yrs, home propose, imagine wait, woman yrs, yrs young, propose woman, husband return, propose, young pron

Topic  5
today change, trust rino, conservative never, discredit trump, change trump, never trust, every conservative, pron everything, continue pron, trust

Topic  6
yes, trump, yes trump, back, pron want, want, pron, air force, body dc, transfer body

Topic  7
involve, give, president, election, accounting, complicit foment, deserv

Community_10

In [31]:
bag_of_words_10 = tfidf_10.fit_transform(X_10)
lsa_10 = TruncatedSVD(20)
doc_topic_10 = lsa_10.fit_transform(bag_of_words_10)
lsa_10.explained_variance_ratio_

array([0.10104706, 0.07003953, 0.03796956, 0.04223051, 0.03818715,
       0.02236901, 0.02465911, 0.01868463, 0.0189153 , 0.01787809,
       0.01596609, 0.01416978, 0.01266458, 0.01072196, 0.00967455,
       0.00884844, 0.0077876 , 0.00748038, 0.00689016, 0.00620884])

In [32]:
display_topics(lsa_10, tfidf_10.get_feature_names(), 10)


Topic  0
save health, entire adult, adult life, life help, people disgraceful, help save, care million, million people, adult, realdonaldtrump continue

Topic  1
always, always shame, lqc mcgmh, lqc, co lqc, senjohnmccain always, mcgmh, always fan, fan senjohnmccain, realdonaldtrump co

Topic  2
enough, tweet attack, tweet, inspirational public, enough inspirational, servant sacrifice, sacrifice much, gqgipdywzx, realdonaldtrump enough, co gqgipdywzx

Topic  3
enough, gqgipdywzx, co gqgipdywzx, enough inspirational, sacrifice much, inspirational public, servant sacrifice, realdonaldtrump enough, inspirational, enough realdonaldtrump

Topic  4
kudo, egfdjv, reintroduce pron, resolution last, pleased co, xoh, sponsor resolution, xoh egfdjv, tribute kudo, pleased

Topic  5
pron, president, self, war hero, friend, war, trump, pron friend, attention, attack depart

Topic  6
self, empathetic, patriotic honorable, sacrifice empathetic, family country, honorable self, country god, friend hero

Community_11

In [33]:
bag_of_words_11 = tfidf_11.fit_transform(X_11)
lsa_11 = TruncatedSVD(20)
doc_topic_11 = lsa_11.fit_transform(bag_of_words_11)
lsa_11.explained_variance_ratio_

array([0.12644582, 0.07579709, 0.05832819, 0.02868003, 0.02595414,
       0.02056228, 0.01723094, 0.01434452, 0.01303006, 0.01161785,
       0.01093918, 0.01099001, 0.01066403, 0.00957572, 0.00846772,
       0.00814263, 0.00808905, 0.00712089, 0.00678499, 0.00645209])

In [34]:
display_topics(lsa_11, tfidf_11.get_feature_names(), 10)


Topic  0
pron, president, xwoiibo, hg, co hg, hg xwoiibo, approve care, care thank, pron ok, fan co

Topic  1
choose pron, choose, pron pron, pron, test never, ever end, choice hard, loyalty test, friend kellyanne, pron principle

Topic  2
never, kunhwo, co kunhwo, never co, trump never, fan never, never fan, fan, president trump, co

Topic  3
pron, instead, team, headline sen, team seek, adversary muell, adversary, manufacture instead, coverage pron, instead headline

Topic  4
pron pron, turn, pron, also, hop put, fbi hop, jeopardy, pron jeopardy, instead alert, dossier authority

Topic  5
follow pron, order avoid, example focus, also follow, pron attention, serve attack, disability order, future pron, president future, attack depart

Topic  6
remember, sfpiw, incredible speech, co sfpiw, remember meghan, fanfare, meghan incredible, fanfare funeral, credit remember, remember fanfare

Topic  7
aide, former aide, trump lie, lie, former, pron, repeal replace, replace, repeal, mind nope


Community_12

In [35]:
bag_of_words_12 = tfidf_12.fit_transform(X_12)
lsa_12 = TruncatedSVD(20)
doc_topic_12 = lsa_12.fit_transform(bag_of_words_12)
lsa_12.explained_variance_ratio_

array([0.28535069, 0.0494183 , 0.04790749, 0.03195337, 0.02791916,
       0.01987265, 0.01340569, 0.00967651, 0.00936284, 0.00898643,
       0.0084225 , 0.00732225, 0.0072493 , 0.00668028, 0.00647529,
       0.00615781, 0.00592635, 0.00574501, 0.00529965, 0.00492186])

In [36]:
display_topics(lsa_12, tfidf_12.get_feature_names(), 10)


Topic  0
pron, billion, feud deceased, focus pron, nation focus, agenda, keep nation, pron agenda, pron rather, rather keep

Topic  1
pron, great courage, aohksjgeqe, co aohksjgeqe, memory american, veteran attack, sen friend, attack memory, president fight, man great

Topic  2
pron, dead note, par famous, write fond, coward par, remembrance pron, dodger donald, fond, leader keep, fond remembrance

Topic  3
president, attack depart, also follow, example focus, pron attention, depart war, behavior president, serve attack, president fake, president future

Topic  4
bi frkphc, frkphc, co bi, bi, trump funeral, reminder trump, reminder, funeral co, funeral, co

Topic  5
time, give, kfxfkhh, co kfxfkhh, jr, separate contribution, separate, give co, ahsaklx gr, ahsaklx

Topic  6
smear, smear attack, must stop, therefore repugnant, smear pron, continuously, pron continuously, apologize hard, therefore, trump smear

Topic  7
pron, aide, funeral, lie, cathedral, former aide, trump lie, former,

Community_13

In [37]:
bag_of_words_13 = tfidf_13.fit_transform(X_13)
lsa_13 = TruncatedSVD(20)
doc_topic_13 = lsa_13.fit_transform(bag_of_words_13)
lsa_13.explained_variance_ratio_

array([0.17618321, 0.10003196, 0.0802793 , 0.03356087, 0.02598526,
       0.02159846, 0.02092684, 0.01898224, 0.01630273, 0.01520078,
       0.01307238, 0.00941866, 0.00816834, 0.00780229, 0.00750061,
       0.00742914, 0.00659331, 0.00594933, 0.00590404, 0.00587636])

In [38]:
display_topics(lsa_13, tfidf_13.get_feature_names(), 10)


Topic  0
republican, republican colin, anthem also, kaepernick disrespectful, disrespectful military, colin kaepernick, veteran kneel, also republican, kneel anthem, kneel

Topic  1
takeaway, takeaway pron, pron beef, past lose, beef trump, spend past, argument dead, lose argument, argument, beef

Topic  2
vjkikfqsb, co vjkikfqsb, trump today, fan trump, today never, trump co, never fan, fan, today, never

Topic  3
pron, shut fuck, fuck, whine, shut, pron country, serve, country, chickenshit serve, fuck forever

Topic  4
pron, president, attention, focus, late, president future, serve attack, also follow, depart war, example focus

Topic  5
billion, pron, funeral, keep, every petty, keep nation, divert attention, attention fact, trump divert, medicare billion

Topic  6
funeral, cathedral, funeral pron, kind funeral, pron want, want, kind, give, president approve, approve

Topic  7
smear, pron, answer smear, therefore repugnant, apologize hard, pron continuously, must stop, hard gop, m

Community_14

In [39]:
bag_of_words_14 = tfidf_14.fit_transform(X_14)
lsa_14 = TruncatedSVD(20)
doc_topic_14 = lsa_14.fit_transform(bag_of_words_14)
lsa_14.explained_variance_ratio_

array([0.15050327, 0.06045983, 0.04722558, 0.04991774, 0.03501545,
       0.02735105, 0.02468907, 0.02131494, 0.01813945, 0.0154613 ,
       0.01551777, 0.01366327, 0.01357324, 0.01202131, 0.01131519,
       0.01045217, 0.00984472, 0.00970974, 0.0090017 , 0.00778447])

In [40]:
display_topics(lsa_14, tfidf_14.get_feature_names(), 10)


Topic  0
pron, cell, dead god, god sake, inch, trash truly, cell beat, spur matter, yacht, lock cell

Topic  1
death certificate, long form, ppfy, senator release, form death, ubrac, co ubrac, ubrac ppfy, call senator, certificate co

Topic  2
still donald, xvhjt, full clip, clip, clip co, lose late, mind full, co xvhjt, senator still, really believe

Topic  3
actually plan, cater macdonald, pron cater, macdonald, plan senator, president actually, cater, actually, plan, senator funeral

Topic  4
peteratlantic, absolutely unacceptable, late absolutely, schwarzenegger call, arnold schwarzenegger, arnold, schwarzenegger, unacceptable, absolutely, attack late

Topic  5
pron, attention, president, focus, order avoid, future, attack depart, also follow, depart war, depart

Topic  6
argue pron, pron person, trump narcissism, lack empathy, empathy argue, degree psychology, psychology trump, argue, psychology, narcissism lack

Topic  7
billion, feud deceased, cut trillion, billion social, bill

Community_15

In [41]:
bag_of_words_15 = tfidf_15.fit_transform(X_15)
lsa_15 = TruncatedSVD(20)
doc_topic_15 = lsa_15.fit_transform(bag_of_words_15)
lsa_15.explained_variance_ratio_

array([0.216564  , 0.07338363, 0.02567562, 0.02409867, 0.01470284,
       0.01431488, 0.01421944, 0.01102413, 0.00999701, 0.00893798,
       0.00825014, 0.00773957, 0.00747898, 0.00685462, 0.006825  ,
       0.00659733, 0.00580656, 0.00559686, 0.00537897, 0.00529608])

In [42]:
display_topics(lsa_15, tfidf_15.get_feature_names(), 10)


Topic  0
cathedral, funeral, president, today funeral, cathedral today, yesterday give, approve washington, sen kind, give sen, government official

Topic  1
pron, client jeffrey, scandal pron, criminal client, moralize, journalist ask, moralize dark, tv moralize, assault scandal, watch baylor

Topic  2
pron, attention, focus, behavior president, serve attack, disability order, future pron, president future, example focus, also follow

Topic  3
billion, every petty, billion social, medicaid billion, deceased pron, feud deceased, rather keep, cut trillion, nation focus, trillion medicaid

Topic  4
never, aide, pron, former aide, trump lie, lie, former, never fan, fan, fan never

Topic  5
white, white supremacist, supremacist, muslim, kill, reminder pron, cable news, walk mosque, week hour, prayer cable

Topic  6
never, never fan, fan never, trump today, fan, co nvnifdhh, nvnifdhh, never trump, today co, co

Topic  7
pron, friend, pron friend, abandon pron, abandon, pron lose, lose pron

Now I have my top words in each LSA object w/ 20 topics each. I'm going to pickle each model and then use a new file to:
 - Find which tweets score the highest in each topic to get examples
 - Name communities
 - Sentiment analysis (maybe weekend)

In [43]:
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_0.pickle', 'wb') as to_write:
    pickle.dump(lsa_0, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_1.pickle', 'wb') as to_write:
    pickle.dump(lsa_1, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_3.pickle', 'wb') as to_write:
    pickle.dump(lsa_3, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_4.pickle', 'wb') as to_write:
    pickle.dump(lsa_4, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_5.pickle', 'wb') as to_write:
    pickle.dump(lsa_5, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_6.pickle', 'wb') as to_write:
    pickle.dump(lsa_6, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_7.pickle', 'wb') as to_write:
    pickle.dump(lsa_7, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_8.pickle', 'wb') as to_write:
    pickle.dump(lsa_8, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_9.pickle', 'wb') as to_write:
    pickle.dump(lsa_9, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_10.pickle', 'wb') as to_write:
    pickle.dump(lsa_10, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_11.pickle', 'wb') as to_write:
    pickle.dump(lsa_11, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_12.pickle', 'wb') as to_write:
    pickle.dump(lsa_12, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_13.pickle', 'wb') as to_write:
    pickle.dump(lsa_13, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_14.pickle', 'wb') as to_write:
    pickle.dump(lsa_14, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_mccain/lsa_15.pickle', 'wb') as to_write:
    pickle.dump(lsa_15, to_write)


NMF Models?

In [44]:
nmf_0 = NMF(20)
doc_topic_NMF_0 = nmf_0.fit_transform(bag_of_words_0)
print(nmf_0.reconstruction_err_)

display_topics(nmf_0, tfidf_0.get_feature_names(), 10)

420.84452677736454

Topic  0
aide, former aide, trump lie, lie, former, pron, replace effort, go support, mind nope, mislead wh

Topic  1
president, president fake, disability order, order avoid, serve attack, focus behavior, example focus, president future, attack depart, also follow

Topic  2
abandon pron, pron lose, abandon, lose pron, lose, pron friend, pron, friend, pow offer, life lindseygrahamsc

Topic  3
cathedral, funeral, national cathedral, national, president, government official, want president, president approve, washington national, government

Topic  4
billion, cut trillion, trillion medicaid, billion medicare, medicare billion, medicaid billion, billion social, budget cut, medicaid, divert attention

Topic  5
smear, repugnant donald, longer answer, therefore repugnant, maliciously trump, continuously maliciously, answer smear, smear attack, apologize hard, pron continuously

Topic  6
soon introduce, forward soon, legislation name, pron legislation, building american, i

In [45]:
nmf_1 = NMF(20)
doc_topic_NMF_1 = nmf_1.fit_transform(bag_of_words_1)
print(nmf_1.reconstruction_err_)

display_topics(nmf_1, tfidf_1.get_feature_names(), 10)

420.6619709338844

Topic  0
aide, former aide, trump lie, lie, former, pron, mind nope, mislead wh, replace effort, effort change

Topic  1
order avoid, disability order, serve attack, president fake, focus behavior, example focus, president future, attack depart, also follow, behavior president

Topic  2
abandon pron, pron lose, abandon, lose pron, lose, pron friend, pron, friend, pow offer, life lindseygrahamsc

Topic  3
cathedral, funeral, national cathedral, national, president, want president, president approve, government official, kind funeral, washington national

Topic  4
billion, trillion medicaid, cut trillion, medicaid billion, medicare billion, billion social, billion medicare, budget cut, medicaid, divert attention

Topic  5
smear, longer answer, repugnant donald, answer smear, therefore repugnant, maliciously trump, continuously maliciously, smear attack, apologize hard, pron continuously

Topic  6
legislation name, soon introduce, forward soon, pron legislation, buildin

In [46]:
nmf_3 = NMF(20)
doc_topic_NMF_3 = nmf_3.fit_transform(bag_of_words_3)
print(nmf_3.reconstruction_err_)

display_topics(nmf_3, tfidf_3.get_feature_names(), 10)

121.27821203086582

Topic  0
smear, pron, trump, therefore repugnant, pron continuously, apologize hard, smear attack, repugnant donald, longer answer, maliciously

Topic  1
pron, whether pron, whether, pron friend, friend, pron pron, slap back, inspiring irascible, friend foe, irascible whether

Topic  2
president, future pron, president future, example focus, also follow, focus behavior, serve attack, disability order, depart war, behavior president

Topic  3
pron, country, towards family, nearly give, record particularly, ahead pron, country ahead, particularly direct, pron less, direct towards

Topic  4
billion, medicaid billion, attention fact, trillion medicaid, pron budget, every petty, cut trillion, budget cut, budget, medicare billion

Topic  5
cathedral, funeral, national cathedral, want president, government official, president, president approve, government, kind funeral, require approval

Topic  6
aide, former aide, trump lie, lie, former, aide add, mislead wh, mind nope, 

In [47]:
nmf_4 = NMF(20)
doc_topic_NMF_4 = nmf_4.fit_transform(bag_of_words_4)
print(nmf_4.reconstruction_err_)

display_topics(nmf_4, tfidf_4.get_feature_names(), 10)

185.8740887770584

Topic  0
pron, obsess, obsess obsess, life spend, friend obsess, never live, weekend pron, live co, family pron, pron life

Topic  1
pron, thank pron, thank, kind funeral, kind, give, pron want, want, pron kind, give pron

Topic  2
pron, president fake, example focus, focus behavior, president future, depart war, also follow, behavior president, attack depart, serve attack

Topic  3
pron pron, pron, great man, great, man, weekend obsess, never great, man co, pron spend, tweet

Topic  4
republican, fire back, back trump, house presidency, cheapen white, silence complicity, trump cheapen, party enable, complicity republican, cheapen

Topic  5
aide, former aide, trump lie, lie, former, pron, pron surprised, effort change, surprised trump, go support

Topic  6
self, empathetic drive, duty family, efface self, drive duty, empathetic, sacrifice empathetic, courageous patriotic, patriotic honorable, heroic courageous

Topic  7
pron, pron pron, dominate news, pron dominate, 

In [48]:
nmf_5 = NMF(20)
doc_topic_NMF_5 = nmf_5.fit_transform(bag_of_words_5)
print(nmf_5.reconstruction_err_)

display_topics(nmf_5, tfidf_5.get_feature_names(), 10)

81.44147736690383

Topic  0
pron, ejtb axp, axp, ejtb, co ejtb, complain come, nice funeral, pron hold, pron nice, hold pron

Topic  1
tempt ghost, ghost twitter, tempt, trump bait, twitter account, bait, account trump, account, twitter, ghost

Topic  2
drag senator, exhume, senator exhume, behind motorcade, ramp, ramp attack, attack drag, exhume corpse, trump ramp, corpse behind

Topic  3
decide, court, number, pron, decide number, court decide, already decide, haggle number, gop already, number seat

Topic  4
fight, vo, hero fight, send american, youth fight, co vo, pron solve, intervention real, pron strongly, solve invasion

Topic  5
ask, many area, public policy, impact, consider ask, yet question, policy impact, want consider, impact people, area

Topic  6
hat, president hat, hat white, hat pron, white nationalist, nationalist, pron hat, white, president, pron

Topic  7
pron pron, pron, die nothing, conservative medium, republican politic, go shit, pee dossier, medium star, polit

In [49]:
nmf_6 = NMF(20)
doc_topic_NMF_6 = nmf_6.fit_transform(bag_of_words_6)
print(nmf_6.reconstruction_err_)

display_topics(nmf_6, tfidf_6.get_feature_names(), 10)

47.99574553257303

Topic  0
pron, president, focus behavior, serve attack, pron attention, depart, senator also, future pron, future, depart war

Topic  1
co wztlcvztnx, state retweet, standard stuff, attack daughter, wztlcvztnx, daughter standard, stuff co, retweet attack, standard, stuff

Topic  2
funeral, thank, thank pron, funeral pron, want, pron want, give, kind funeral, pron, kind

Topic  3
isakson, johnny isakson, johnny, play politic, effort rename, politic effort, schumer play, co ielln, cme, ielln

Topic  4
aide, former aide, trump lie, former, lie, replace, repeal replace, repeal, mislead wh, go support

Topic  5
smear, repugnant donald, longer answer, pron apologize, apologize hard, maliciously, trump smear, answer smear, smear attack, pron therefore

Topic  6
pron pron, pron, trump, co, turn, call, pron call, dossier, friend, fbi

Topic  7
pron legislation, soon introduce, forward soon, look forward, legislation name, senate russell, introduce pron, building american, nam

In [50]:
nmf_7 = NMF(20)
doc_topic_NMF_7 = nmf_7.fit_transform(bag_of_words_7)
print(nmf_7.reconstruction_err_)

display_topics(nmf_7, tfidf_7.get_feature_names(), 10)

99.09434616882511

Topic  0
always denigrate, today always, unseemly trump, denigrate widow, widow lindsay, anything unseemly, graham nothing, trump constantly, constantly attack, denigrate

Topic  1
implication, infuriate many, expose pron, wonder implication, implication guilty, haunt sadden, sense poetic, insecurity wonder, sadden infuriate, poetic justice

Topic  2
disgusting ananavarro, hero plain, dead national, plain disgusting, wr lcgmm, indecent immoral, everybody condemn, immoral everybody, lcgmm, co wr

Topic  3
aide, former aide, trump lie, lie, former, pron, happen former, aide add, mind nope, mislead wh

Topic  4
disability order, example focus, serve attack, focus behavior, future pron, behavior president, also follow, president future, president fake, attack depart

Topic  5
give holiday, scrooge instead, holiday turkey, holiday, haunt crap, trump happen, happen scrooge, back haunt, man give, resign turn

Topic  6
fear inadequacy, lt col, male, inadequacy, retire lt, ap

In [51]:
nmf_8 = NMF(20)
doc_topic_NMF_8 = nmf_8.fit_transform(bag_of_words_8)
print(nmf_8.reconstruction_err_)

display_topics(nmf_8, tfidf_8.get_feature_names(), 10)

109.35131151150915

Topic  0
president seriously, seriously stop, stop talk, seriously, stop, talk senator, mr president, mr, talk, senator

Topic  1
self, friend heroic, drive duty, heroic courageous, duty family, empathetic drive, efface self, courageous patriotic, sacrifice empathetic, self efface

Topic  2
poltical, poltical enemy, enemy mr, pron poltical, reason talk, pass pron, pron, pron pass, enemy, pass

Topic  3
pron, service legacy, admiration peace, everyone give, hero thankful, country arizona, arizona everyone, family respect, respect admiration, thankful pron

Topic  4
pron, lot never, tear reputation, country idea, anything patriot, willing lay, hero particularly, patriot willing, feel tear, reputation war

Topic  5
every, pron, hero, memory continue, sustain, continue remind, every miss, hero senate, rare patriot, blessing serve

Topic  6
hit, man, pron hit, late mark, patriotic man, president pick, man hit, memory president, lose saint, repellent

Topic  7
pron, serve

In [52]:
nmf_9 = NMF(20)
doc_topic_NMF_9 = nmf_9.fit_transform(bag_of_words_9)
print(nmf_9.reconstruction_err_)

display_topics(nmf_9, tfidf_9.get_feature_names(), 10)

389.51038720690843

Topic  0
pron, pron pron, trust rino, today change, conservative never, discredit trump, change trump, never trust, every conservative, pron everything

Topic  1
fbi, december senator, agent bill, confirm fbi, priestap confirm, copy first, fbi receive, receive copy, senior counterintelligence, counterintelligence fbi

Topic  2
fake, pron, paper last, annapolis send, indeed prove, court paper, prove court, medium hop, print election, together fail

Topic  3
medium, racist divisive, divisive voice, advance narrative, use advance, voice medium, narrative trump, medium use, medium attack, hate donald

Topic  4
young, pron, yrs, woman yrs, imagine wait, home propose, yrs young, propose woman, husband return, propose

Topic  5
bad, rino, pron, pron pron, angry president, rino angry, center bitter, insufferable self, pron insufferable, bitter rino

Topic  6
yes, yes trump, trump, help win, force transfer, transfer body, az race, body dc, announce pron, yes back

Topic  7
g

In [53]:
nmf_10 = NMF(20)
doc_topic_NMF_10 = nmf_10.fit_transform(bag_of_words_10)
print(nmf_10.reconstruction_err_)

display_topics(nmf_10, tfidf_10.get_feature_names(), 10)

76.2921931518059

Topic  0
people disgraceful, care million, save health, adult life, life help, entire adult, help save, million people, adult, realdonaldtrump continue

Topic  1
always, mcgmh, always fan, fan senjohnmccain, always shame, lqc, lqc mcgmh, co lqc, senjohnmccain always, realdonaldtrump co

Topic  2
tweet attack, tweet, snl fox, clinton paris, obama doj, democrat hillary, zealand shooting, cia pron, shooting realdonaldtrump, anchor muell

Topic  3
enough, sacrifice much, enough inspirational, realdonaldtrump enough, co gqgipdywzx, inspirational public, servant sacrifice, inspirational, gqgipdywzx, enough realdonaldtrump

Topic  4
co xoh, pleased, xoh egfdjv, egfdjv, kudo senator, pleased co, sponsor resolution, xoh, fitting tribute, tribute kudo

Topic  5
president, also follow, president future, order avoid, attack depart, example focus, depart, depart war, serve attack, president fake

Topic  6
self, empathetic, duty family, family country, sacrifice empathetic, courage

In [54]:
nmf_11 = NMF(20)
doc_topic_NMF_11 = nmf_11.fit_transform(bag_of_words_11)
print(nmf_11.reconstruction_err_)

display_topics(nmf_11, tfidf_11.get_feature_names(), 10)

75.38324813119145

Topic  0
pron, president, co hg, xwoiibo, hg xwoiibo, hg, approve care, care thank, pron ok, fan co

Topic  1
choose pron, choose, pron pron, pron, test never, conway choose, end choice, husband loyalty, pron principle, graham choose

Topic  2
never, kunhwo, co kunhwo, never co, trump never, fan never, never fan, fan, president trump, president

Topic  3
pron, tout manufacture, script, muell pron, pron team, save lima, effort save, tout, ohio promote, foreign adversary

Topic  4
pron pron, pron, turn, hop put, fbi hop, pron jeopardy, jeopardy, campaign instead, nb co, mblvt nb

Topic  5
pron, order avoid, attack depart, serve attack, focus behavior, president future, also follow, attention pron, follow pron, depart war

Topic  6
remember, speech co, fanfare funeral, incredible speech, sfpiw, meghan incredible, remember meghan, co sfpiw, remember fanfare, fanfare

Topic  7
aide, former aide, trump lie, lie, former, pron, ex aide, support repeal, surprised trump, repla

In [55]:
nmf_12 = NMF(20)
doc_topic_NMF_12 = nmf_12.fit_transform(bag_of_words_12)
print(nmf_12.reconstruction_err_)

display_topics(nmf_12, tfidf_12.get_feature_names(), 10)

73.26584195065841

Topic  0
pron, billion, pron agenda, nation focus, pron rather, feud deceased, keep nation, focus pron, agenda, rather keep

Topic  1
veteran attack, great courage, memory american, aohksjgeqe, co aohksjgeqe, sen friend, attack memory, president fight, man great, friend man

Topic  2
pron, famous draft, remembrance pron, coward par, fond remembrance, fond, dead note, par, par famous, write fond

Topic  3
president, president fake, focus behavior, depart war, attack depart, behavior president, serve attack, example focus, pron attention, president future

Topic  4
bi frkphc, co bi, frkphc, bi, trump funeral, reminder trump, reminder, funeral co, funeral, co

Topic  5
time, give, ahsaklx, gr, limit prompt, limit, kfxfkhh co, campaign refund, separate, prompt campaign

Topic  6
smear, trump smear, therefore, repugnant donald, smear attack, answer smear, continuously maliciously, longer answer, apologize hard, pron continuously

Topic  7
pron, pron pron, funeral, thank, 

In [56]:
nmf_13 = NMF(20)
doc_topic_NMF_13 = nmf_13.fit_transform(bag_of_words_13)
print(nmf_13.reconstruction_err_)

display_topics(nmf_13, tfidf_13.get_feature_names(), 10)

56.29235108081401

Topic  0
republican, republican colin, anthem also, disrespectful military, colin kaepernick, veteran kneel, kneel anthem, kaepernick disrespectful, also republican, colin

Topic  1
pron beef, takeaway pron, takeaway, past lose, spend past, beef trump, argument dead, lose argument, argument, beef

Topic  2
co vjkikfqsb, vjkikfqsb, trump today, fan trump, today never, trump co, never fan, fan, never, today

Topic  3
shut fuck, fuck, whine, shut, pron, bullshit excuse, excuse imaginary, quack, quack doctor, nice poltroon

Topic  4
pron, president, follow pron, serve attack, example focus, disability order, focus behavior, attack depart, depart war, future pron

Topic  5
billion, pron, billion medicare, medicare billion, medicaid billion, medicaid, billion social, pron budget, nation focus, divert attention

Topic  6
cathedral, funeral, president, national cathedral, national, government official, government, want president, cathedral require, require approval

Topic  7

In [57]:
nmf_14 = NMF(20)
doc_topic_NMF_14 = nmf_14.fit_transform(bag_of_words_14)
print(nmf_14.reconstruction_err_)

display_topics(nmf_14, tfidf_14.get_feature_names(), 10)

52.13385940535964

Topic  0
pron, matter pron, god sake, cell, listen trump, cell beat, disturb guy, within inch, lock cell, truly disturb

Topic  1
pron long, call senator, senator release, death certificate, form death, co ubrac, ubrac ppfy, ubrac, certificate co, long form

Topic  2
clip, lose late, full clip, xvhjt, co xvhjt, still donald, senator still, clip co, mind full, trump beef

Topic  3
plan senator, cater macdonald, pron cater, macdonald, actually plan, president actually, cater, actually, plan, senator funeral

Topic  4
peril online, gov trump, co xbjadjasn, xbjadjasn, consult pron, bullying co, trump consult, peteratlantic former, wife peril, calif

Topic  5
president, pron, attack depart, order avoid, example focus, depart war, president future, president fake, disability order, future

Topic  6
argue, trump narcissism, lack empathy, narcissism lack, empathy argue, psychology, psychology trump, argue pron, pron person, degree psychology

Topic  7
billion, pron, budget c

In [58]:
nmf_15 = NMF(20)
doc_topic_NMF_15 = nmf_15.fit_transform(bag_of_words_15)
print(nmf_15.reconstruction_err_)

display_topics(nmf_15, tfidf_15.get_feature_names(), 10)

95.39387714706673

Topic  0
cathedral, funeral, president, today funeral, approve washington, yesterday give, cathedral today, sen kind, give sen, government official

Topic  1
moralize, scandal pron, moralize dark, watch baylor, assault scandal, journalist ask, least journalist, client jeffrey, starr go, criminal client

Topic  2
serve attack, president future, depart war, attack depart, behavior president, disability order, focus behavior, also follow, future pron, order avoid

Topic  3
billion, attention fact, feud deceased, pron rather, keep nation, deceased pron, medicaid billion, every petty, pron budget, cut trillion

Topic  4
aide, former aide, trump lie, lie, former, happen former, senator mislead, lie ex, claim today, aide trump

Topic  5
prayer, prayer cable, reminder pron, supremacist walk, zealand kill, hour since, kill muslim, walk mosque, mosque zealand, exactly week

Topic  6
never, never fan, fan never, trump today, co nvnifdhh, nvnifdhh, never trump, today co, fan, co

In [59]:
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_0.pickle', 'wb') as to_write:
    pickle.dump(nmf_0, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_1.pickle', 'wb') as to_write:
    pickle.dump(nmf_1, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_3.pickle', 'wb') as to_write:
    pickle.dump(nmf_3, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_4.pickle', 'wb') as to_write:
    pickle.dump(nmf_4, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_5.pickle', 'wb') as to_write:
    pickle.dump(nmf_5, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_6.pickle', 'wb') as to_write:
    pickle.dump(nmf_6, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_7.pickle', 'wb') as to_write:
    pickle.dump(nmf_7, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_8.pickle', 'wb') as to_write:
    pickle.dump(nmf_8, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_9.pickle', 'wb') as to_write:
    pickle.dump(nmf_9, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_10.pickle', 'wb') as to_write:
    pickle.dump(nmf_10, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_11.pickle', 'wb') as to_write:
    pickle.dump(nmf_11, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_12.pickle', 'wb') as to_write:
    pickle.dump(nmf_12, to_write)

with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_13.pickle', 'wb') as to_write:
    pickle.dump(nmf_13, to_write)

with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_14.pickle', 'wb') as to_write:
    pickle.dump(nmf_14, to_write)

with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_mccain/nmf_15.pickle', 'wb') as to_write:
    pickle.dump(nmf_15, to_write)
    

Running Topic Modeling on ALL text data, both NMF and LSA, on various topic counts, to feed into DBScan / KMeans for cluster analysis

In [60]:
# ## Creating list of the processed text files to merge, so it's all in the same format and can re-use code
# dfs_to_merge = [df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_26, df_40]

In [61]:
# total = 0

# for df in dfs_to_merge:
#     print(df.shape)
#     total += df.shape[0]
    
# print(total)

In [62]:
# df_all_preprocessed = pd.concat(dfs_to_merge)

In [63]:
# df_all_preprocessed.shape

In [64]:
# df_all = df_all_preprocessed.copy()

In [65]:
# df_all.head()

In [66]:
# with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_all.pickle', 'wb') as to_write:
#     pickle.dump(df_all, to_write)

In [67]:
# with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_all.pickle', 'rb') as f:
#     df_all = pickle.load(f)

So the concat worked, the # of rows equals the number of all the rows added up (just to be safe)

Now I'll run a few different LSAs and NMFs on the total data, and move on to clustering

# Preprocess Text - for both LSA and NMF

In [68]:
# # Can come back to tweek this if I notice anything
# tweet_stopwords = stopwords.words('english') + \
#     ['rt', 'https', 'http', 'amp', 'via', 'one', 'around', 'would', 'let', 'could', 'going', 'like', 
#      'get', 'may', 'says', 'say', 'make', 'based', 'even', 'another', 'completely', 'thanks', 'way', 
#      'find', 'used', 'thing', '2019', 'see', 'need', 'know', 'knows', 'think', 'thinks', 'take', 'new', 
#      'day', 'days', 'aoc', 'alexandria', 'ocasio', 'cortez', 'ocasio-cortez', 'pron']

In [69]:
# tfidf_all = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))

In [70]:
# X_all = df_all['tweet_processed']

In [71]:
# bag_of_words_all = tfidf_all.fit_transform(X_all)

In [72]:
# type(bag_of_words_all)

# LSA on all text
*10, 15, 20, 25 topics*

In [73]:
# # LSA on 10 topics
# lsa_all_10 = TruncatedSVD(10)
# lsa_all_10.fit_transform(bag_of_words_all)
# lsa_all_10.explained_variance_ratio_

In [74]:
# display_topics(lsa_all_10, tfidf_all.get_feature_names(), 10)

In [75]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_10.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_10, to_write)

---------

In [76]:
# # LSA on 15 topics
# lsa_all_15 = TruncatedSVD(15)
# lsa_all_15.fit_transform(bag_of_words_all)
# lsa_all_15.explained_variance_ratio_

In [77]:
# display_topics(lsa_all_15, tfidf_all.get_feature_names(), 10)

In [78]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_15.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_15, to_write)

---------

In [79]:
# # LSA on 20 topics
# lsa_all_20 = TruncatedSVD(20)
# lsa_all_20.fit_transform(bag_of_words_all)
# lsa_all_20.explained_variance_ratio_

In [80]:
# display_topics(lsa_all_20, tfidf_all.get_feature_names(), 10)

In [81]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_20.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_20, to_write)

---------

In [82]:
# # LSA on 25 topics
# lsa_all_25 = TruncatedSVD(25)
# lsa_all_25.fit_transform(bag_of_words_all)
# lsa_all_25.explained_variance_ratio_

In [83]:
# display_topics(lsa_all_25, tfidf_all.get_feature_names(), 10)

In [84]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_25.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_25, to_write)

# NMF on all text
*10, 15, 20, 25 topics*

In [85]:
# # NMF on 10 topics
# nmf_all_10 = NMF(10)
# nmf_all_10.fit_transform(bag_of_words_all)
# print(nmf_all_10.reconstruction_err_)

In [86]:
# display_topics(nmf_all_10, tfidf_all.get_feature_names(), 10)

In [87]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_10.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_10, to_write)

---------

In [88]:
# # NMF on 15 topics
# nmf_all_15 = NMF(15)
# nmf_all_15.fit_transform(bag_of_words_all)
# print(nmf_all_15.reconstruction_err_)

In [89]:
# display_topics(nmf_all_15, tfidf_all.get_feature_names(), 10)

In [90]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_15.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_15, to_write)

---------

In [91]:
# # NMF on 20 topics
# nmf_all_20 = NMF(20)
# nmf_all_20.fit_transform(bag_of_words_all)
# print(nmf_all_20.reconstruction_err_)

In [92]:
# display_topics(nmf_all_20, tfidf_all.get_feature_names(), 10)

In [93]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_20.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_20, to_write)

---------

In [94]:
# # NMF on 25 topics
# nmf_all_25 = NMF(25)
# nmf_all_25.fit_transform(bag_of_words_all)
# print(nmf_all_25.reconstruction_err_)

# ## setting a variable to the fit transformed

In [95]:
# nmf_model_25_df = pd.DataFrame(data=nmf_all_25.fit_transform(bag_of_words_all))
# nmf_model_25_df.shape

In [96]:
# print(nmf_all_25.reconstruction_err_)
# nmf_model_25_df.head()

NOTE TO SELF - if the above works, I'll need to make DFs of all versions again (which will take around another hour) :(

Then I'll concat them with index and screen name, so I can assign values back to the tweeters otherwise. One annoying thing is I didn't pass in tweet ID so I'll have to think about that too

In [97]:
# display_topics(nmf_all_25, tfidf_all.get_feature_names(), 10)

In [98]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_25.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_25, to_write)

Now I will create DF's where I append this info to a dataframe

then try dbscan with these

In [99]:
# df_topic_base = df_all.reset_index()
# df_topic_base.head()

crap - now I know why I saved the fit_transform(bag_of_words_ objects previously, that's what I need to use to create the df. For now I'm seeing if I can just call that as the "data" and see if that works. Otherwise I will need to set them all equal to variables

In [100]:
# df_topic_base.shape

In [101]:
# lsa_all_10 = TruncatedSVD(10)

# lsa_model_10_df = pd.DataFrame(data=lsa_all_10.fit_transform(bag_of_words_all))
# lsa_model_10_df.shape

In [102]:
# lsa_all_15 = TruncatedSVD(15)

# lsa_model_15_df = pd.DataFrame(data=lsa_all_15.fit_transform(bag_of_words_all))
# lsa_model_15_df.shape

In [103]:
# lsa_all_20 = TruncatedSVD(20)

# lsa_model_20_df = pd.DataFrame(data=lsa_all_20.fit_transform(bag_of_words_all))
# lsa_model_20_df.shape

In [104]:
# lsa_all_25 = TruncatedSVD(25)

# lsa_model_25_df = pd.DataFrame(data=lsa_all_25.fit_transform(bag_of_words_all))
# lsa_model_25_df.shape

In [105]:
# df_lsa_model_10 = pd.merge(df_topic_base, lsa_model_10_df, left_index=True, right_index=True)
# df_lsa_model_15 = pd.merge(df_topic_base, lsa_model_15_df, left_index=True, right_index=True)
# df_lsa_model_20 = pd.merge(df_topic_base, lsa_model_20_df, left_index=True, right_index=True)
# df_lsa_model_25 = pd.merge(df_topic_base, lsa_model_25_df, left_index=True, right_index=True)

---------

In [106]:
# nmf_all_10 = NMF(10)

# nmf_model_10_df = pd.DataFrame(data=nmf_all_10.fit_transform(bag_of_words_all))
# nmf_model_10_df.shape

In [107]:
# nmf_all_15 = NMF(15)

# nmf_model_15_df = pd.DataFrame(data=nmf_all_15.fit_transform(bag_of_words_all))
# nmf_model_15_df.shape

In [108]:
# nmf_all_20 = NMF(20)

# nmf_model_20_df = pd.DataFrame(data=nmf_all_20.fit_transform(bag_of_words_all))
# nmf_model_20_df.shape

In [109]:
# # nmf_model_25_df = pd.DataFrame(data=nmf_all_25.fit_transform(bag_of_words_all)) ALREADY DONE!
# nmf_model_25_df.shape

In [110]:
# df_nmf_model_10 = pd.merge(df_topic_base, nmf_model_10_df, left_index=True, right_index=True)
# df_nmf_model_15 = pd.merge(df_topic_base, nmf_model_15_df, left_index=True, right_index=True)
# df_nmf_model_20 = pd.merge(df_topic_base, nmf_model_20_df, left_index=True, right_index=True)
# df_nmf_model_25 = pd.merge(df_topic_base, nmf_model_25_df, left_index=True, right_index=True)


In [112]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_10.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_10, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_15.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_15, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_20.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_20, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_25.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_25, to_write)
    

# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_10.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_10, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_15.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_15, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_20.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_20, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_25.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_25, to_write)
    
    

    
